In [1]:
import sqlite3
import pandas as pd
import numpy as np

### Basic Query

In [ ]:
SELECT * 
FROM table
LIMIT 10;

In [ ]:
# in SQL lite
conn = sqlite3.connect('nba_db')
cur = conn.cursor()

In [ ]:
cur.execute("""SELECT *,
               FROM players
               LIMIT 10;""")

### LIKE

In [ ]:
SELECT last_name, height
FROM players
WHERE last_name ILIKE 'jam&';

### IN

In [ ]:
SELECT *
FROM games
WHERE month IN ('May', 'June');


### NOT

In [ ]:
SELECT *
FROM games
WHERE month NOT IN ('May', 'June');


### AND

In [ ]:
SELECT team,
FROM games
WHERE month NOT IN ('May', 'June')
AND home_score <= 100;


### OR

In [ ]:
SELECT *
FROM players
WHERE team LIKE ‘%Lakers’
AND (last_name = ‘Davis’ OR last_name = ‘James);


### BETWEEN

In [ ]:
SELECT *
FROM games
WHERE date BETWEEN '2020-01-01' AND '2020-04-01';


### NULL

In [ ]:
SELECT *
FROM players
WHERE last_name IS NULL;


### Simple Join

In [ ]:
# player points
SELECT p.last_name, g.points
FROM player p
JOIN game g
ON g.player_id = p.id
ORDER BY g.points DESC;

### Double Join

In [ ]:
# team points
SELECT t.name, g.points
FROM player p
JOIN game g
ON g.player_id = p.id
JOIN team t
ON t.player_id = p.id
ORDER BY g.points;


### Date Part

In [ ]:
# Sum of Laker points from October to August
# No games between 03/11 and 7/29
SELECT DATE_PART('month', g.game_time) month, SUM(g.points) points
FROM player p
JOIN game g
ON g.player_id = p.id
JOIN team t
ON t.player_id = p.id
WHERE t.name LIKE '%Lakers'
AND g.game_time BETWEEN '2019-10-01' AND '2020-08-13'
GROUP BY 1
ORDER BY 1;

### HAVING

In [ ]:
# players who have scored over 1000 points for their team
SELECT t.name, p.last_name, SUM(g.points) points
FROM player p
JOIN game g
ON g.player_id = p.id
JOIN team t
ON t.player_id = p.id
GROUP BY t.name, p.last_name
HAVING SUM(g.points) > 1000
ORDER BY points DESC;

### CASE

In [ ]:
# LeBron points after restart
SELECT p.last_name, t.name, g.id, SUM(g.points) points,
     CASE WHEN SUM(g.points) > 40 THEN 'top'
     WHEN  SUM(g.points) > 20 THEN 'middle'
     ELSE 'low' END AS scoring_level
FROM player p
JOIN game g
ON g.player_id = p.id
JOIN team t
ON t.player_id = p.id
WHERE p.last_name = 'James'
AND g.game_time > '2019-07-29'
GROUP BY 1,2,3
ORDER BY 4 DESC;

#### What is a subquery

What is a subquery? Very simply, it is a query inside of a query. They are useful when you need to carry out a process multiple steps. Often, to retrieve the information you want, you have to do some work on your data before selecting, filtering, or calculating. Subqueries are a popular way of performing this work.

### WHERE clause

In [ ]:
SELECT
-- Select the team names
    team_name    
FROM team
-- Filter for teams with 130 or more point
WHERE team_id IN
      (SELECT team_id 
       FROM game
       WHERE points >= 130);

### FROM clause

In [ ]:
SELECT
-- Select player, date, and points from the subquery
    player_name,
    date,
    points,
FROM 
    -- Subquery: select player name, date, and points
    (SELECT p.last_name AS player_name, 
            g.date, 
           (g.field_goal + g.three_pt + g.free_throw) AS points
    FROM game AS g
    LEFT JOIN player AS p
    ON g.player_id = p.id) AS subquery
-- Filter main query by total points scored
WHERE points >= 40;


### SELECT clause

In [ ]:
SELECT
-- Select the team name and average points scored
    t.name AS team,
    ROUND(AVG(g.field_goal + g.three_pt + g.free_throw),0) 
AS avg_point,
    -- Subtract the overall average from the team average
    ROUND(AVG(g.field_goal + g.three_pt + g.free_throw) - 
        (SELECT AVG(g.field_goal + g.three_pt + g.free_throw)
         FROM game 
         WHERE season = '2019/2020'),0) AS diff
FROM team AS t
LEFT JOIN game AS g
ON t.id = g.team_id
-- Filter to 2019/2020 season
WHERE season = '2019/2020'
GROUP BY t.name;